In [5]:
import numpy as np
import sys
import pandas as pd 

In [4]:
#смотрел, в чём разница генерации, не важно
weights_1_2 = np.random.normal(0.0, 1, (1, 20))
weights_1_3 = np.random.rand(1, 20)
print(weights_1_2)
print(weights_1_3)

[[ 0.09337725 -0.86783209  0.15816484 -0.66411524  0.66192892  2.31557162
   0.25822253  0.21084051  0.69331964  0.74436449 -0.58980105  0.94484288
   0.05150788  1.64698237  0.80491978 -0.44704634  2.78171149 -0.5213339
   0.7684103  -0.76854913]]
[[0.22955442 0.84480895 0.73157692 0.0380451  0.78092887 0.301893
  0.99219128 0.45246019 0.00467052 0.60992717 0.42350226 0.077756
  0.43791898 0.9339905  0.72053641 0.82195951 0.40752005 0.374686
  0.1050577  0.0646559 ]]


#Нейронная сеть

In [37]:
class IsDiabet(object):
    
    #структура нейросети
    def __init__(self, learning_rate=0.1):
        #веса: 2 - кол во нейронов в hidden_layer_1, 3 - кол-во нейронов на входе 
        self.weights_0_1 = np.random.normal(0.0, 2 ** -0.5, (20, 21)) 
        #веса: 1 - кол-во нейронов на выходе,  2 - кол во нейронов в hidden_layer_1 - вход 
        self.weights_1_2 = np.random.normal(0.0, 1, (1, 20))
        #self.weights_1_2 = np.random.rand(1, 20)
        #применение сигмоиды ко всем элементам вектора
        self.sigmoid_mapper = np.vectorize(self.sigmoid)
        #коэффициент для обучения
        self.learning_rate = np.array([learning_rate])
    #функция активации
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))


    #вычисление взвешенной суммы весов 
    def w_sum(weights, inputs):
      inputs_1 = np.dot(weights, inputs)
      return inputs_1

    #прямое распространение (предсказание)
    def predict(self, inputs):
        #получение значений нейронов в hidden_layer_1 - взвешенная сумма
        inputs_1 = np.dot(self.weights_0_1, inputs)
        outputs_1 = self.sigmoid_mapper(inputs_1)
        #outputs_1 = w_sum(self.weights_0_1, inputs)
        
        #получение значений нейрона на выходе
        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2)
        return outputs_2

    #тренировка
    def train(self, inputs, expected_predict):     
        inputs_1 = np.dot(self.weights_0_1, inputs)
        #значения нейронов скрытого слоя
        outputs_1 = self.sigmoid_mapper(inputs_1)

        
        inputs_2 = np.dot(self.weights_1_2, outputs_1)
        outputs_2 = self.sigmoid_mapper(inputs_2) #значение на выходе
        # предсказание для текущих входных данных
        actual_predict = outputs_2[0]
        # actual_predict = self.predict(inputs)
        
        #значение ошибки
        error_layer_2 = np.array([actual_predict - expected_predict])
        #градиент - производная от сигмоиды
        gradient_layer_2 = actual_predict * (1 - actual_predict)
        #деьлта весов - то, на сколько их нужно изменить
        weights_delta_layer_2 = error_layer_2 * gradient_layer_2 
        #пересчёт весов от  hidden_layer_1 на выход
        self.weights_1_2 -= (np.dot(weights_delta_layer_2, outputs_1.reshape(1, len(outputs_1)))) * self.learning_rate

        #пересчёт весов от  входа на hidden_layer_1
        error_layer_1 = weights_delta_layer_2 * self.weights_1_2
        gradient_layer_1 = outputs_1 * (1 - outputs_1)
        weights_delta_layer_1 = error_layer_1 * gradient_layer_1
        self.weights_0_1 -= np.dot(inputs.reshape(len(inputs), 1), weights_delta_layer_1).T  * self.learning_rate

In [10]:
#средняя квадратичная ошибка
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [ ]:
#считывание данных из файла
df = pd.read_csv('diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
#создайм массив входных данных
dataset = df.to_numpy()
#перемешиваем строки, иначе что-то странное
np.random.shuffle(dataset)
#df
#df.dtypes


In [ ]:
#проверял работу срезов
print(dataset)
print(dataset[:,0])
print (len(dataset[:,0]))
X, Y = dataset[0:,1:22], dataset[0:,0]
print (X)
print (Y)
print("***")
X, Y = dataset[i,1:22], dataset[i,0]
print (X)
print (Y)

##обучение

In [51]:

epochs = 15 #сколько раз будем проходить по входным данным
learning_rate = 0.01

#создание нейросети
network = IsDiabet(learning_rate=learning_rate)

#обучение нейросети
for e in range(epochs):
    # разбиваем датасет на матрицу входных данных (X) и вектор ожидаемых значений переменной (Y)
    X, Y = dataset[:,1:22], dataset[:,0]
    inputs_ = []
    correct_predictions = []
    l = len(dataset[:,0])
    right_predictions = 0
    for i in range (l):
        input_stat = dataset[i,1:22]
        correct_predict = dataset[i,0]
        network.train(np.array(input_stat), correct_predict)
        inputs_.append(np.array(input_stat))
        correct_predictions.append(np.array(correct_predict))
        
        #смотрим, на сколько ошиблись
        our_predict = network.predict(dataset[i,1:22])[0]
        if our_predict >= 0.5 and correct_predict == 1:
          right_predictions +=1
        elif our_predict < 0.5 and correct_predict == 0:
          right_predictions +=1
    accuracy = right_predictions/l    
   
    train_loss = MSE(network.predict(np.array(inputs_).T), np.array(correct_predictions))
    print("Iteration: " + str(e) + ";  Accuracy: " + str(accuracy) + ";  Loss: " + str(train_loss) + "\n")

Iteration: 0;  Accuracy: 0.7132065863181124;  Loss: 0.18976926850103853

Iteration: 1;  Accuracy: 0.7641883098511855;  Loss: 0.18727450567924678

Iteration: 2;  Accuracy: 0.7684886550104679;  Loss: 0.18416738863524987

Iteration: 3;  Accuracy: 0.7678096531432128;  Loss: 0.1839425605038766

Iteration: 4;  Accuracy: 0.7670740677870197;  Loss: 0.183221287126454

Iteration: 5;  Accuracy: 0.7687291348384542;  Loss: 0.1822734923792847

Iteration: 6;  Accuracy: 0.7688564476885644;  Loss: 0.1820402557183949

Iteration: 7;  Accuracy: 0.7691393651332541;  Loss: 0.18213303470018247

Iteration: 8;  Accuracy: 0.7688281559440955;  Loss: 0.18182471781045004

Iteration: 9;  Accuracy: 0.7685028008827024;  Loss: 0.18018187556509013

Iteration: 10;  Accuracy: 0.7686159678605783;  Loss: 0.1791487784850785

Iteration: 11;  Accuracy: 0.7683613421603576;  Loss: 0.17938438520367625

Iteration: 12;  Accuracy: 0.7677672155265094;  Loss: 0.17951469149692875

Iteration: 13;  Accuracy: 0.7678520907599162;  Loss: 0